In [14]:
'''
1. Do the number of songs for each year/artist have to be roughly equal for training?
2. Embed words given the line that they are in? independent from rest of lyrics
3. Remove or keep/use section tags, e.g [Chorus]
'''
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
import requests, json
from bs4 import BeautifulSoup
import re

url = 'https://en.wikipedia.org/wiki/List_of_hip_hop_musicians'
page = requests.get(url)
aList = []
pattern = r"(<li><a href=\"\/wiki\/)(.*?)(\" title=\")(.*?)(\">)(.*?)(<\/a><\/li>)"
for x in re.finditer(pattern, page.text, re.S):
    artist = x.group(4)
    if artist == "List of hip hop groups":
        break
    if "(" in artist:
        artist = artist.split("(")[0][:-1]
        aList.append(artist)
    else:
        for char in artist:
            if not char.isalnum() and char != " ":
                artist = artist.replace(char,"")
        aList.append(artist)
#print(aList)

In [10]:
GENIUS_URL = "https://api.genius.com"
CLIENT_ACCESS_TOKEN = "Enter Client Key"
HEADERS = {"Authorization": "Bearer {}".format(CLIENT_ACCESS_TOKEN)}

def gen_get(path, params = None, headers = None):

    url = '/'.join([GENIUS_URL, path])
    
    response = requests.get(url=url, params=params, headers=HEADERS)
    response.raise_for_status()

    return response.json()

def get_hits(artist_id):

    curr = 5
    next = True
    songs = []

    while next:

        path = "artists/{}/songs/".format(artist_id)
        params = {'page': curr}
        data = gen_get(path=path, params=params, headers = HEADERS)
        pages = data['response']['songs']
        if pages and curr < 10:
            songs += pages
            curr += 1
        else:
            next = False

    songs = [str(song["id"]) for song in songs if song["primary_artist"]["id"] == artist_id]
    return songs

# Populate
full = []

# Truncated aList for testing
for i,name in enumerate(aList):
    if i % int(len(aList) / 20) == 0:
        print(i,"/",len(aList))
    find_id = gen_get("search", {'q': name})
    for hit in find_id["response"]["hits"]:
        if hit["result"]["primary_artist"]["name"] == name:
            artist_id = hit["result"]["primary_artist"]["id"]
            break
    hits = get_hits(artist_id)
    song_ids = hits

    
    full.append(song_ids)

0 / 1315
65 / 1315
130 / 1315
195 / 1315
260 / 1315
325 / 1315
390 / 1315
455 / 1315
520 / 1315
585 / 1315
650 / 1315
715 / 1315
780 / 1315
845 / 1315
910 / 1315
975 / 1315
1040 / 1315
1105 / 1315
1170 / 1315
1235 / 1315
1300 / 1315


In [1]:
# setup for collecting song lyrics

import requests, json, threading
from bs4 import BeautifulSoup

GENIUS_URL = "https://api.genius.com"
CLIENT_ACCESS_TOKEN = "jTG-fZt7TV6PAVEuxFTlcMZz7eMRQCMuvaHRDlXZq8NgkuU4OlHIi3GN6AFy9FAD"
HEADERS = {"Authorization": "Bearer {}".format(CLIENT_ACCESS_TOKEN)}

if not full:
    full = []
    with open("artists_combined", "r", encoding="utf8") as artist_songs:
        for line in artist_songs:
            line = line.strip()
            full.append([l for l in line.split(',') if l is not ''][1:])

d = {}

def get_lyrics(artists, id_to_song):
    print("Starting Thread ({})".format(threading.current_thread().name), flush=True)
    try:
        for i,artist_songs in enumerate(artists):
            j = -1
            if len(artists) >= 10 and i % int(len(artists) / 10) == 0:
                print("Thread ({})".format(threading.current_thread().name), i,"/",len(artists), flush=True)
            # Truncated song list for testing
            for j,song in enumerate(artist_songs):
                api_url = GENIUS_URL + "/songs/" + song
                response = requests.get(api_url, headers=HEADERS)
                json_body = response.json()
                
                if json_body["meta"]["status"] != 200:
                    continue
                # value: [song title, artist name, artist id, release year]
                song_title = json_body["response"]["song"]["title"]
                artist_name = json_body["response"]["song"]["primary_artist"]["name"]
                artist_id = json_body["response"]["song"]["primary_artist"]["id"]
                release_date = json_body["response"]["song"]["release_date"]
                # Check if release date is available
                if release_date:
                    year = int(release_date[:4])
                    # Get lyrics
                    song_url = json_body["response"]["song"]["url"]
                    request = requests.get(song_url, headers=HEADERS)
                    page = BeautifulSoup(request.text, "html.parser")
                    [l.extract() for l in page('script')]
                    lyrics = page.find("div", class_="lyrics").get_text()

                    id_to_song[song] = {"title": song_title, "artist": artist_name, "id" : artist_id, "year": year, "lyrics": lyrics}                
    except Exception as e:
        print("Thread ({})".format(threading.current_thread().name),"Error", str(e), "occured at artist", i, "song", j,  flush=True)

In [3]:
# Collect all the lyrics for every single song in full

threads = []

# Number of threads to use (more = faster)
num_threads = 10

step = int(len(full) / num_threads)

for i in range(0, len(full), step):
    artist_slice = full[i:i+step]
    if len(artist_slice) == 0:
        continue
    threads.append(threading.Thread(target=get_lyrics, args=(artist_slice, d), name=str(i)))

for t in threads:
    t.start()

print("Waiting for threads",  flush=True)
    
for t in threads:
    t.join()

print("All done writing to file now",  flush=True)

# Output to file in json format
with open("songs.json", "w+") as out_file:
    json.dump(d, out_file)

print("Completely finished", flush=True)

Starting Thread (0)
Starting Thread (131)
Starting Thread (262)
Starting Thread (393)
Thread (131) 0 / Thread (262)Thread (0) 0 131
 0 / 131
/ 131
Starting Thread (524)
Starting Thread (655)
Thread (655) 0 / 131
Thread (524) 0 / 131
Starting Thread (786)Thread (393)
Thread (786) 0 / 131
Starting Thread (917)
Thread (917) 0 / 131
 0 / 131
Starting Thread (1048)
Starting Thread (1179)
Thread (1048) 0 / 131
Starting Thread (1310)Waiting for threads

Thread (1179) 0 / 131
Thread (917) 13 / 131
Thread (262) 13 / 131
Thread (786) 13 / 131
Thread (524) 13 / 131
Thread (0) 13 / 131
Thread (393) 13 / 131
Thread (131) 13 / 131
Thread (1179) 13 / 131
Thread (917) 26 / 131
Thread (1048) 13 / 131
Thread (786) 26 / 131
Thread (655) 13 / 131
Thread (0) 26 / 131
Thread (393) 26 / 131
Thread (524) 26 / 131
Thread (1179) 26 / 131
Thread (786) 39 / 131
Thread (262) 26 / 131
Thread (917) 39 / 131
Thread (655) 26 / 131
Thread (131) 26 / 131
Thread (1048) 26 / 131
Thread (393) 39 / 131
Thread (0) 39 / 131
T

In [2]:
# Finding the lowest year
import json
with open("songs.json", "r") as json_file:
    songs = json.loads(json_file.read())
    min_year = 2030
    for song in songs:
        year = songs[song]["year"]
        if year < min_year and year > 1900:
            print(year, songs[song]["title"], songs[song]["artist"])
            min_year = songs[song]["year"]
print(min_year)

2018 All Me Jeremiah Jae
2017 03 Purple Hearts 03 Greedo
2005 100 Degrees Fort Minor
2003 Back Again Juelz Santana
1997 Party Ain't A Party Queen Pen
1996 Fox Boogie Foxy Brown
1995 About Me Raekwon
1994 Gonna Give It To Ya Jewell
1967 Song Rampage
1967
